# Funções
Arquivo de funções de machine Learning para prever boas apostas de futebol para a Bet365 na temporada 2021/2022 da Premier League

In [126]:
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import re
import glob

In [127]:
# Lista de caminhos para os arquivos CSV de cada temporada
paths = glob.glob('../data/seasons/*.csv')

# Lista para armazenar os datasets de cada temporada
datasets = []

# Carregar os datasets de cada temporada
for path in paths:
    # Extrair o nome da temporada do caminho do arquivo
    temporada = path.split('/')[-1].split('.')[0]
    
    # Carregar o dataset da temporada e adicionar a coluna 'Temporada'
    dataset = pd.read_csv(path)
    dataset['Temporada'] = temporada
    
    # Adicionar o dataset à lista
    datasets.append(dataset)

# Concatenar os datasets em um único dataframe
data = pd.concat(datasets, ignore_index=True)

# Substituir valores ausentes por valores nulos (NaN)
data.replace('?', pd.NA, inplace=True)

# Exibir o dataframe resultante
print(data.head())


  Div      Date  HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  ...  \
0  E0  19/08/00  Charlton       Man City     4     0   H     2     0   H  ...   
1  E0  19/08/00   Chelsea       West Ham     4     2   H     1     0   H  ...   
2  E0  19/08/00  Coventry  Middlesbrough     1     3   A     1     1   D  ...   
3  E0  19/08/00     Derby    Southampton     2     2   D     1     2   A  ...   
4  E0  19/08/00     Leeds        Everton     2     0   H     2     0   H  ...   

   AvgC<2.5 AHCh  B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  AvgCAHH  \
0       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
1       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
2       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
3       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   
4       NaN  NaN       NaN       NaN    NaN    NaN      NaN      NaN      NaN   

   AvgCAHA  
0      NaN  


In [128]:
# Filtrar os resultados antes de 31/06/2021
train_data = data[data['Date'] < '2021-06-30']

# Selecionar as colunas relevantes para treinamento
train_data = train_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
train_data_encoded = pd.get_dummies(train_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_train = train_data_encoded.drop("FTR", axis=1)
y_train = train_data_encoded["FTR"]

# Filtrar os resultados após 31/06/2021
test_data = data[data['Date'] > '2021-06-30']

# Selecionar as colunas relevantes para teste
test_data = test_data[["HomeTeam", "AwayTeam", "FTHG", "FTAG", "HS", "AS", "HST", "AST", "FTR"]]

# Converta as variáveis categóricas em variáveis numéricas usando one-hot encoding
test_data_encoded = pd.get_dummies(test_data, columns=["HomeTeam", "AwayTeam"], drop_first=True)

# Separar as features (variáveis de entrada) e o target (variável de saída)
X_test = test_data_encoded.drop("FTR", axis=1)
y_test = test_data_encoded["FTR"]

# Crie e ajuste o modelo de regressão logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Crie e ajuste o modelo de árvore de decisão
decision_tree = DecisionTreeClassifier(random_state=42)
decision_tree.fit(X_train, y_train)

# Crie e ajuste o modelo de Random Forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Crie e ajuste o modelo SVM
svm = SVC()
svm.fit(X_train, y_train)

# Crie e ajuste o modelo Naive Bayes
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)

# Avalie a acurácia dos modelos
accuracy_logreg = logreg.score(X_test, y_test)
accuracy_decision_tree = decision_tree.score(X_test, y_test)
accuracy_random_forest = random_forest.score(X_test, y_test)
accuracy_svm = svm.score(X_test, y_test)
accuracy_naive_bayes = naive_bayes.score(X_test, y_test)

# Imprima as acurácias
print("Acurácia (Regressão Logística):", accuracy_logreg)
print("Acurácia (Árvore de Decisão):", accuracy_decision_tree)
print("Acurácia (Random Forest):", accuracy_random_forest)
print("Acurácia (SVM):", accuracy_svm)
print("Acurácia (Naive Bayes):", accuracy_naive_bayes)

print(train_data_encoded)


c:\Users\guifl\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Acurácia (Regressão Logística): 1.0
Acurácia (Árvore de Decisão): 0.9989785495403473
Acurácia (Random Forest): 0.992849846782431
Acurácia (SVM): 0.9976166155941437
Acurácia (Naive Bayes): 0.5175348995573714
      FTHG  FTAG  HS  AS  HST  AST FTR  HomeTeam_Aston Villa  \
0        4     0  17   8   14    4   H                     0   
1        4     2  17  12   10    5   H                     0   
2        1     3   6  16    3    9   A                     0   
3        2     2   6  13    4    6   D                     0   
4        2     0  17  12    8    6   H                     0   
...    ...   ...  ..  ..  ...  ...  ..                   ...   
8005     2     0  16  11    4    2   H                     0   
8006     1     2   4  24    2    5   A                     0   
8007     3     2  12   7    6    5   H                     0   
8008     1     1  22  10    9    5   D                     1   
8009     1     1  20   2    7    1   D                     0   

      HomeTeam_Birmingha

In [129]:
def obter_estatisticas_confronto(home_team, away_team):
    # Filtrar os jogos anteriores entre os times
    confrontos = data[((data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)) |
                      ((data['HomeTeam'] == away_team) & (data['AwayTeam'] == home_team))]

    # Calcular as estatísticas dos confrontos
    total_jogos = confrontos.shape[0]
    vitorias_home = confrontos[confrontos['HomeTeam'] == home_team]['FTR'].eq('H').sum()
    vitorias_away = confrontos[confrontos['AwayTeam'] == home_team]['FTR'].eq('A').sum()
    empates = confrontos[confrontos['FTR'] == 'D'].shape[0]

    # Criar DataFrame com as estatísticas
    estatisticas = pd.DataFrame({'TotalJogos': [total_jogos],
                                 'VitoriasHome': [vitorias_home],
                                 'VitoriasAway': [vitorias_away],
                                 'Empates': [empates]})

    return estatisticas


In [136]:
def obter_medias_times(home_team, away_team):
    # Filtrar os jogos do time
    jogos = data[(data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team)]

    # Calcular as médias de gols, chutes e chutes no alvo
    home_media_gols = jogos['FTHG'].mean()
    home_media_chutes = jogos['HS'].mean()
    home_media_chutes_alvo = jogos['HST'].mean()
    away_media_gols = jogos['FTAG'].mean()
    away_media_chutes = jogos['AS'].mean()
    away_media_chutes_alvo = jogos['AST'].mean()

    # Criar DataFrame com as médias
    medias = pd.DataFrame({'HomeTeam': [home_team],
                            'AwayTeam': [away_team],
                            'FTHG': [home_media_gols],
                            'HS': [home_media_chutes],
                            'HST': [home_media_chutes_alvo],
                            'FTAG': [away_media_gols],
                            'AS': [away_media_chutes],
                            'AST': [away_media_chutes_alvo]})

    # Adicionar colunas de codificação dos times
    for col in X_train.columns:
        if col.startswith("HomeTeam_") and col != f"HomeTeam_{home_team}":
            medias[col] = 0
        elif col.startswith("AwayTeam_") and col != f"AwayTeam_{away_team}":
            medias[col] = 0
        elif col.startswith("HomeTeam_") and col == f"HomeTeam_{home_team}":
            medias[col] = 1
        elif col.startswith("AwayTeam_") and col == f"AwayTeam_{away_team}":
            medias[col] = 1
    
    # Reordenar as colunas
    medias = medias[X_train.columns]
    

    # Adicionar colunas ausentes
    missing_cols = set(X_train.columns) - set(medias.columns)
    for col in missing_cols:
        medias[col] = 0

    return medias


In [168]:
db = pathlib.Path.cwd() / "../data/seasons/2021-2022.csv"
data = pd.read_csv(db)

# Função para fazer uma sugestão de aposta com base no modelo treinado
def fazer_sugestao_aposta(jogo, modelo):
    jogo_encoded = obter_medias_times(jogo['HomeTeam'], jogo['AwayTeam'])

    # Fazer a previsão com base no modelo treinado
    predicao = modelo.predict(jogo_encoded)
    probabilidade = modelo.predict_proba(jogo_encoded)

    # Filtrar o jogo específico
    jogo_especifico = data[(data['HomeTeam'] == jogo['HomeTeam']) & (data['AwayTeam'] == jogo['AwayTeam'])]

    # Obter as odds específicas do jogo
    odd_home = jogo_especifico['B365H'].values[0]
    odd_draw = jogo_especifico['B365D'].values[0]
    odd_away = jogo_especifico['B365A'].values[0]

    # Criar o dicionário com a sugestão de aposta
    sugestao_aposta = {
        'HomeTeam': jogo['HomeTeam'],
        'AwayTeam': jogo['AwayTeam'],
        'Predicao': predicao[0],
        'OddHome': odd_home,
        'OddDraw': odd_draw,
        'OddAway': odd_away,
    }

    # Se a previsão for de vitória do time da casa
    if predicao[0] == 'H':
        sugestao = "Sugestão de aposta: Vitória do time da casa com odd de {}".format(odd_home)
    # Se a previsão for de empate
    elif predicao[0] == 'D':
        sugestao = "Sugestão de aposta: Empate com odd de {}".format(odd_draw)
    # Se a previsão for de vitória do time visitante
    else:
        sugestao = "Sugestão de aposta: Vitória do time visitante com odd de {}".format(odd_away)

    return sugestao

# Carregar o modelo treinado
modelo = random_forest

# Exemplo de jogo específico
jogo = {
    "HomeTeam": 'Man United',
    "AwayTeam": 'Man City',
}

# Fazer a sugestão de aposta com base no modelo treinado
sugestao_aposta = fazer_sugestao_aposta(jogo, modelo)

print("Sugestão de aposta:", sugestao_aposta)


Sugestão de aposta: Sugestão de aposta: Vitória do time visitante com odd de 1.7
